In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from imblearn.over_sampling import  SMOTE


import tensorflow as tf


In [2]:
df_cleaned=pd.read_csv("/content/drive/MyDrive/Project-4 Group 6/post_clean.csv")
df_cleaned

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.89,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,Female,80.0,1,0,Yes,Private,Urban,83.75,28.89,never smoked,0
5105,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.00,never smoked,0
5106,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.60,never smoked,0
5107,Male,51.0,0,0,Yes,Private,Rural,166.29,25.60,formerly smoked,0


In [3]:
df_cleaned['gender'].value_counts()

Female    2994
Male      2115
Name: gender, dtype: int64

In [4]:
df_new= pd.get_dummies(df_cleaned)
df_new.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.60,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0
1,61.0,0,0,202.21,28.89,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0
2,80.0,0,1,105.92,32.50,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.40,1,1,0,0,1,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.00,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0


In [6]:
y= df_new['stroke']
X= df_new.drop(columns="stroke")

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=1)

In [8]:
# Creating Random Over Sampler to balance dataset
smote= SMOTE(sampling_strategy=.2,random_state=9)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [9]:
# # Testing to see if this can be done?
X_test_resampled, y_test_resampled = smote.fit_resample(X_test, y_test)

In [10]:
pd.Series(y_train_resampled).value_counts(normalize=True)

0    0.833486
1    0.166514
Name: stroke, dtype: float64

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_resampled)

# Scale the data
X_train_r_scaled = X_scaler.transform(X_train_resampled)


In [12]:
# x_test does not get resampled (Is that necessary)? 
X_test_r_scaled = X_scaler.transform(X_test_resampled)

In [13]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_dimen= len(X_train_r_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= 10,activation='relu',input_dim= input_dimen))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= 8,activation='leaky_relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units= 1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                200       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [15]:
# Train the model
fit_model= nn.fit(X_train_r_scaled,y_train_resampled,epochs=100)

Epoch 1/100
137/137 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7376
Epoch 2/100
137/137 [==============================] - 0s 3ms/step - loss: 0.3730 - accuracy: 0.8422
Epoch 3/100
137/137 [==============================] - 0s 3ms/step - loss: 0.2621 - accuracy: 0.9179
Epoch 4/100
137/137 [==============================] - 0s 3ms/step - loss: 0.1980 - accuracy: 0.9419
Epoch 5/100
137/137 [==============================] - 0s 3ms/step - loss: 0.1719 - accuracy: 0.9483
Epoch 6/100
137/137 [==============================] - 0s 3ms/step - loss: 0.1615 - accuracy: 0.9495
Epoch 7/100
137/137 [==============================] - 1s 7ms/step - loss: 0.1556 - accuracy: 0.9527
Epoch 8/100
137/137 [==============================] - 1s 7ms/step - loss: 0.1533 - accuracy: 0.9504
Epoch 9/100
137/137 [==============================] - 1s 4ms/step - loss: 0.1500 - accuracy: 0.9531
Epoch 10/100
137/137 [==============================] - 0s 3ms/step - loss: 0.1484 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_r_scaled,y_test_resampled,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

46/46 - 0s - loss: 0.1679 - accuracy: 0.9527 - 220ms/epoch - 5ms/step
Loss: 0.1678590178489685, Accuracy: 0.9527073502540588


In [17]:
roc_auc_score(y_test_resampled,nn.predict(X_test_r_scaled))

46/46 [==============================] - 0s 3ms/step


0.9412192711717565